# Qcodes example with Agilent 34400A

In [1]:
%matplotlib nbagg
import matplotlib.pyplot as plt
import time
import numpy as np

import qcodes as qc
from qcodes.utils.validators import Enum, Strings
import qcodes.instrument_drivers.tektronix.Keithley_2600 as keith
import qcodes.instrument_drivers.agilent.Agilent_34400A as agi



<IPython.core.display.Javascript object>

In [2]:
import time
class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        if self.name:
            print('[%s]' % self.name,)
        print('Elapsed: %s' % (time.time() - self.tstart))

In [3]:
# create Instruments
k1 = keith.Keithley_2600('Keithley1', 'GPIB0::15::INSTR',channel='a')
k2 = keith.Keithley_2600('Keithley2', 'GPIB0::15::INSTR',channel='b')

a1 = agi.Agilent_34400A('Agilent1', 'GPIB0::11::INSTR')
a2 = agi.Agilent_34400A('Agilent2', 'GPIB0::6::INSTR')

# set integration time (number of line cycles)
a1.NPLC.set(10)
a2.NPLC.set(10)

In [4]:
station1 = qc.Station(a1,a2)
station1.set_measurement(a1.volt)
station2 = qc.Station(a1,a2)
station2.set_measurement(a1.volt, a2.volt)

In [5]:
# Time single readings
with Timer('Time s1'):
    station1.measure()
with Timer('Time s2'):
    station2.measure()

[Time s1]
Elapsed: 2.0011143684387207


Empty: 

In [ ]:
# Time single readings
with Timer('Time a1'):
    a1.volt.get()
with Timer('Time a2'):
    a2.volt.get()

In [ ]:
with Timer('Time Loop 1'):
    data = qc.Loop(k1.volt[-5:5:1], 0).each(a1.volt).run(location='testsweep', overwrite=True,background=False)

with Timer('Time Loop 2'):
    data = qc.Loop(k1.volt[-5:5:1], 0).each(a1.volt, a2.volt).run(location='testsweep', overwrite=True,background=False)

In [ ]:
with Timer('Time Loop 1'):
    data = qc.Loop(k1.volt[-5:5:1], 0).each(a1.volt).run(location='testsweep', overwrite=True)
    while data.sync():
        time.sleep(0.1)

In [ ]:
with Timer('Time Loop 2'):
    data = qc.Loop(k1.volt[-5:5:1], 0).each(a1.volt, a2.volt).run(location='testsweep', overwrite=True)
    while data.sync():
        time.sleep(0.1)